# 学習（SSD）

準備（変換）した学習データを用いて、実際に SSD (Single Shot Multibox Detector) で学習を実行します。

## 概要

機械学習の「学習」フェーズでは、学習モデルと入力データを準備し、パラメータを設定し、そして実際に実行（イテレーション実施）、という手順を踏んでいきます。  
このノートブックでは、そういった手順を体験してみます。

学習モデルは SSD (Single Shot Multibox Detector)、入力は前のステップで用意した学習データを利用します。

## 準備

In [ ]:
import sys
sys.path.append(r'../models/research')
sys.path.append(r'../models/research/slim')

import functools
import glob
import json
import re

import tensorflow as tf

In [ ]:
from object_detection import trainer
from object_detection.builders import dataset_builder
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import dataset_util

↑警告が出ることがありますが気にしないでください。

## 設定読込

モデルおよび学習（訓練）の設定は、まとめて `ssd_mobilenet_v1_manabiya.pbtxt` というファイルに記述してあります。これを TF Object Detection で用意されているユーティリティ関数を利用して読み込みます。

In [ ]:
configs = config_util.get_configs_from_pipeline_file('./ssd_mobilenet_v1_manabiya.pbtxt')

In [ ]:
model_config = configs['model']
train_config = configs['train_config']
input_config = configs['train_input_config']

### モデル設定確認

In [ ]:
model_config

In [ ]:
model_config.ssd.num_classes

### 訓練 設定確認

In [ ]:
train_config

In [ ]:
train_config.num_steps

↑ デフォルトは2000ステップにしてありますが、所要時間目安は以下の通り：

+ `ml.p3.2xlarge`（GPUインスタンス）で実行した場合、約18分で完了します。  
+ `ml.p2.xlarge`（GPUインスタンス）で実行した場合、約48分で完了します。  
+ `ml.m4.xlarge`（CPUインスタンス）で実行した場合、約10時間（！）で完了します。  

ハンズオン時間内に終わるよう、適宜修正してください。

参考までに、所要時間が15分となるステップ数の目安を以下に示します：

+ `ml.p3.2xlarge`（GPUインスタンス）：1600ステップ前後
+ `ml.p2.xlarge`（GPUインスタンス）：600ステップ前後
+ `ml.m4.xlarge`（CPUインスタンス）：40ステップ前後

In [ ]:
# train_config.num_steps = 100

In [ ]:
train_config.num_steps

## 学習

TF Object Detection で用意されている訓練機能を利用して、学習を実行します。

### モデル

`model_fn` を準備します。これは、学習に利用するモデルを生成する関数オブジェクトです。

In [ ]:
model_fn = functools.partial(
    model_builder.build,
    model_config=model_config,
    is_training=True)

### 入力

`create_input_dict_fn` を準備します。これは、学習に利用する入力テンソル（教師およびラベル）を生成する関数オブジェクトです。

In [ ]:
def get_next(config):
    return dataset_util.make_initializable_iterator(
        dataset_builder.build(config)).get_next()

create_input_dict_fn = functools.partial(get_next, input_config)

### その他学習パラメータ

TF Object Detection では、マルチタスク・マルチノードでの学習（分散学習）をすることもできます。  
今回は簡単のため、シングルタスク・シングルノードで学習を実施します。以下はそのための設定となります。

In [ ]:
# シングルタスクの設定
cluster_data = None
cluster = None
task_data = {'type': 'master', 'index': 0}
task_info = type('TaskSpec', (object,), task_data)

In [ ]:
# シングルノードの設定
ps_tasks = 0
worker_replicas = 1
worker_job_name = 'lonely_worker'
task = 0
is_chief = True
master = ''
num_clones = 1
clone_on_cpu = False

In [ ]:
# 学習結果（チェックポイントファイル、データ確認用サマリ）出力先
train_dir = './train'

### 学習（訓練）実行

In [ ]:
%%time
trainer.train(create_input_dict_fn, model_fn, train_config, master, task,
    num_clones, worker_replicas, clone_on_cpu, ps_tasks,
    worker_job_name, is_chief, train_dir)

↑最初に警告が出ますが気にしないでください。

## 補足：学習経過の観察

実際の学習フェーズでは、学習（訓練）実行時に、並行してその経過を随時確認したりします。  
その主な目的は以下の通りです：

+ 学習の「進み具合」を確認する
    + 例：loss の値を確認して発散していないかを確認する
+ 「過学習」が起きていないかを確認する
    + 例：訓練精度と検証精度を評価・比較して、乖離が起きていないかを確認する
+ 学習の「やめどき」を図る
    + 例：訓練精度と検証精度の乖離が起き始めた（＝過学習しだした）らやめる
    + 例：評価指標を1つ決めて、その値が頭打ちになった（それ以上精度向上が見込めなくなった）らやめる

学習の「進み具合」に関しては、今回も「学習（訓練）実行」セルの出力に随時 loss を表示しています。  
「過学習」（および「やめどき」）に関しては今回のハンズオンでは省きます。  

なお実際の現場では、ログ出力を目視確認するより、可視化（折れ線グラフ表示などで確認）ことも多いです。  
例えば TensorFlow には「[TensorBoard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard)」という可視化ツールが用意されています。TF Object Detection での学習結果の出力ファイルにはこの TensorBoard で可視化できる形式のデータも含まれています。

## 補足：追加学習

今回は限られた時間内のハンズオンなので、少ないステップ数での学習しか実行していません。  

TF Object Detection は、学習途中からの追加学習にも対応しています。  
このノートブックで言えば、上記の「学習（訓練）実行」セルを再実行すれば、それまでの学習結果を読み込んで学習を継続します（例えば、100ステップまで進んでいたら101ステップ目から継続します）。

必要に応じて `train_config.num_steps` を調整して、またCPUインスタンスで実行していた場合はGPUインスタンスに切り替えたりして、追加学習も試してみてください。

## 補足：SageMaker を利用した学習（および推測）について

今回はハンズオン時間とライブラリやモデルの都合上、ノートブックインスタンス上で学習（およびその後の推測（検出））をすべて行っています。

SageMaker では学習や推測のために別インスタンスを建てて、ノートブックインスタンスとは独立して実行・実施することも出来ます。  

+ ジョブ：
    + 学習時に建てられるインスタンス
    + 学習経過を随時確認することも出来る（TensorFlow 利用時は [TensorBoard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard) によるグラフのリアルタイム閲覧も）
    + 指定したイテレーションが完了したら、S3にモデル（の学習後の変数情報）がアップロードされ、自動停止することもできる
+ エンドポイント
    + アップロードされたモデルを元に推測を行うためのインスタンス
    + 所謂デプロイ。WebAPIを提供する

特にデプロイしてエンドポイントを建てると、「画像を送って何が写っているか返す」ようなアプリに組み込んで使うことも出来るようになります。  